## Weather Harvester 

In [1]:
!pip install pymysql==1.0.2

You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.


In [2]:
# sql alchemy 
import sqlalchemy as sqla 
from sqlalchemy import create_engine 

# tracing for exception hendeling 
import traceback 

# to follow unix files and paths 
import glob 

# fucntions for interacting with the operating system 
import os 

# send http requests using python 
import requests 

# represents time in code 
import time 

#format json objects 
import json 

#file with privite 
import my_secrets


from IPython.display import display

# mysql driver
import pymysql

import requests

import traceback

import datetime

import time

import os

from my_secrets import APIKEY # NB I PUT API KEY IN my_secrets AS WELL

In [3]:
DB_USER = os.getenv('DB_USER')
DB_PASS = os.environ.get('DB_PASS')
DB_URL = os.environ.get('DB_URL')
DUBBIKE_API = os.getenv('DUBBIKE_API')
DUBBIKE_NAME = os.environ.get('DUBBIKE_NAME')
DUBBIKE_STATIONS = os.environ.get('DUBBIKE_STATIONS')
WAPIKEY = os.environ.get('WAPIKEY')

print(WAPIKEY)

74571a8a54ecc764ebed7d0dc725ebe2


In [4]:
print(DB_PASS)

comp30830_ec2


In [5]:
for a in os.environ:
    print('Var: ', a, 'Value: ', os.getenv(a))


Var:  SHELL Value:  /bin/bash
Var:  KUBERNETES_SERVICE_PORT_HTTPS Value:  443
Var:  JUPYTER_CONFIG_DIR Value:  /deepnote-config/jupyter
Var:  KUBERNETES_SERVICE_PORT Value:  443
Var:  HOSTNAME Value:  p-71d679fb-aa0e-4eb1-84cc-5c32f888c0ea
Var:  PYTHON_VERSION Value:  3.8
Var:  LANGUAGE Value:  en_US.UTF-8
Var:  WAPIKEY Value:  74571a8a54ecc764ebed7d0dc725ebe2
Var:  GIT_SSH_COMMAND Value:  ssh -i /work/.deepnote/gitkey -o UserKnownHostsFile=/dev/null -o StrictHostKeyChecking=no
Var:  PWD Value:  /
Var:  JUPYTER_TOKEN Value:  fb218641-c625-4f84-8737-ce77313592ad
Var:  PLOTLY_RENDERER Value:  colab
Var:  IPYTHONDIR Value:  /deepnote-config/ipython
Var:  DB_USER Value:  ec2
Var:  HOME Value:  /root
Var:  LANG Value:  en_US.UTF-8
Var:  KUBERNETES_PORT_443_TCP Value:  tcp://10.100.0.1:443
Var:  VIRTUAL_ENV Value:  /root/venv
Var:  DEEPNOTE_PROJECT_ID Value:  71d679fb-aa0e-4eb1-84cc-5c32f888c0ea
Var:  DUBBIKE_STATIONS Value:  https://api.jcdecaux.com/vls/v1/stations/
Var:  GPG_KEY Value:  E3

Connecting to DB

In [6]:
!pip install pymysql==1.0.2

You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.


In [7]:
engine = create_engine("mysql+pymysql://{0}:{1}@{2}".format(DB_USER
, DB_PASS, DB_URL), echo=True)
connection = engine.connect()

2021-02-25 12:58:41,056 INFO sqlalchemy.engine.base.Engine SHOW VARIABLES LIKE 'sql_mode'
2021-02-25 12:58:41,058 INFO sqlalchemy.engine.base.Engine {}
2021-02-25 12:58:41,063 INFO sqlalchemy.engine.base.Engine SHOW VARIABLES LIKE 'lower_case_table_names'
2021-02-25 12:58:41,064 INFO sqlalchemy.engine.base.Engine {}
2021-02-25 12:58:41,070 INFO sqlalchemy.engine.base.Engine SELECT DATABASE()
2021-02-25 12:58:41,070 INFO sqlalchemy.engine.base.Engine {}
2021-02-25 12:58:41,079 INFO sqlalchemy.engine.base.Engine show collation where `Charset` = 'utf8mb4' and `Collation` = 'utf8mb4_bin'
2021-02-25 12:58:41,080 INFO sqlalchemy.engine.base.Engine {}
2021-02-25 12:58:41,084 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS CHAR(60)) AS anon_1
2021-02-25 12:58:41,085 INFO sqlalchemy.engine.base.Engine {}
2021-02-25 12:58:41,086 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS CHAR(60)) AS anon_1
2021-02-25 12:58:41,087 INFO sqlalchemy.engine.base.E

In [8]:
#check if db is running 
for res in engine.execute('SHOW VARIABLES;'): 
    print(res)

2021-02-25 12:58:41,107 INFO sqlalchemy.engine.base.Engine SHOW VARIABLES;
2021-02-25 12:58:41,108 INFO sqlalchemy.engine.base.Engine {}
('activate_all_roles_on_login', 'OFF')
('admin_address', '')
('admin_port', '33062')
('auto_generate_certs', 'ON')
('auto_increment_increment', '1')
('auto_increment_offset', '1')
('autocommit', 'OFF')
('automatic_sp_privileges', 'ON')
('avoid_temporal_upgrade', 'OFF')
('back_log', '66')
('basedir', '/rdsdbbin/mysql-8.0.20.R3/')
('big_tables', 'OFF')
('bind_address', '*')
('binlog_cache_size', '32768')
('binlog_checksum', 'CRC32')
('binlog_direct_non_transactional_updates', 'OFF')
('binlog_encryption', 'OFF')
('binlog_error_action', 'ABORT_SERVER')
('binlog_expire_logs_seconds', '2592000')
('binlog_format', 'MIXED')
('binlog_group_commit_sync_delay', '0')
('binlog_group_commit_sync_no_delay_count', '0')
('binlog_gtid_simple_recovery', 'ON')
('binlog_max_flush_queue_time', '0')
('binlog_order_commits', 'ON')
('binlog_rotate_encryption_master_key_at_sta

In [9]:
# create table - SOUTH 
sql_create_table = """
CREATE TABLE IF NOT EXISTS `dublin_bikes`.`open_weather_south`(
coord_lon REAL,
coord_lat REAL,
weather_id INTEGER,
weather_main VARCHAR(256),
weather_description VARCHAR(256),
weather_icon VARCHAR(256),
base VARCHAR(256),
main_temp FLOAT,
main_feels_like FLOAT,
main_temp_min FLOAT,
main_temp_max FLOAT,
main_pressure FLOAT,
main_humidity INTEGER,
visibility INTEGER,
wind_speed FLOAT,
wind_deg INTEGER,
rain_1h FLOAT,
clouds_all INTEGER,
dt INTEGER,
sys_type INTEGER,
sys_id INTEGER,
sys_country VARCHAR(256),
sys_sunrise INTEGER,
sys_sunset INTEGER,
timezone INTEGER,
id INTEGER,
name VARCHAR(256),
cod INTEGER,
)
"""

try: 
    res = engine.execute(sql_create_table)
    print(res.fetchall())
except Exception as e:
    print(e)

2021-02-25 12:58:41,188 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE IF NOT EXISTS `dublin_bikes`.`open_weather_south`(
coord_lon REAL,
coord_lat REAL,
weather_id INTEGER,
weather_main VARCHAR(256),
weather_description VARCHAR(256),
weather_icon VARCHAR(256),
base VARCHAR(256),
main_temp FLOAT,
main_feels_like FLOAT,
main_temp_min FLOAT,
main_temp_max FLOAT,
main_pressure FLOAT,
main_humidity INTEGER,
visibility INTEGER,
wind_speed FLOAT,
wind_deg INTEGER,
rain_1h FLOAT,
clouds_all INTEGER,
dt INTEGER,
sys_type INTEGER,
sys_id INTEGER,
sys_country VARCHAR(256),
sys_sunrise INTEGER,
sys_sunset INTEGER,
timezone INTEGER,
id INTEGER,
name VARCHAR(256),
cod INTEGER,
)

2021-02-25 12:58:41,194 INFO sqlalchemy.engine.base.Engine {}
2021-02-25 12:58:41,196 INFO sqlalchemy.engine.base.Engine ROLLBACK
(pymysql.err.ProgrammingError) (1064, "You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near ')' at line 30")
[S

In [10]:
# create table - NORTH
sql_create_table = """
CREATE TABLE IF NOT EXISTS `dublin_bikes`.`open_weather_north`(
coord_lon REAL,
coord_lat REAL,
weather_id INTEGER,
weather_main VARCHAR(256),
weather_description VARCHAR(256),
weather_icon VARCHAR(256),
base VARCHAR(256),
main_temp FLOAT,
main_feels_like FLOAT,
main_temp_min FLOAT,
main_temp_max FLOAT,
main_pressure FLOAT,
main_humidity INTEGER,
visibility INTEGER,
wind_speed FLOAT,
wind_deg INTEGER,
rain_1h FLOAT,
clouds_all INTEGER,
dt INTEGER,
sys_type INTEGER,
sys_id INTEGER,
sys_country VARCHAR(256),
sys_sunrise INTEGER,
sys_sunset INTEGER,
timezone INTEGER,
id INTEGER,
name VARCHAR(256),
cod INTEGER,
)
"""

try: 
    res = engine.execute(sql_create_table)
    print(res.fetchall())
except Exception as e:
    print(e)

2021-02-25 12:58:41,203 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE IF NOT EXISTS `dublin_bikes`.`open_weather_north`(
coord_lon REAL,
coord_lat REAL,
weather_id INTEGER,
weather_main VARCHAR(256),
weather_description VARCHAR(256),
weather_icon VARCHAR(256),
base VARCHAR(256),
main_temp FLOAT,
main_feels_like FLOAT,
main_temp_min FLOAT,
main_temp_max FLOAT,
main_pressure FLOAT,
main_humidity INTEGER,
visibility INTEGER,
wind_speed FLOAT,
wind_deg INTEGER,
rain_1h FLOAT,
clouds_all INTEGER,
dt INTEGER,
sys_type INTEGER,
sys_id INTEGER,
sys_country VARCHAR(256),
sys_sunrise INTEGER,
sys_sunset INTEGER,
timezone INTEGER,
id INTEGER,
name VARCHAR(256),
cod INTEGER,
)

2021-02-25 12:58:41,205 INFO sqlalchemy.engine.base.Engine {}
2021-02-25 12:58:41,206 INFO sqlalchemy.engine.base.Engine ROLLBACK
(pymysql.err.ProgrammingError) (1064, "You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near ')' at line 30")
[S

In [ ]:
def error_log(e):
    """
    Function to record errors to output txt file
    """
    now = datetime.datetime.utcnow()
    try:
        file = open("OWM_Current_log.txt", "x")
    except FileExistsError:
        file = open("OWM_Current_log.txt", "a")
    finally:
        file.write(str(e) + "\t" + str(now.strftime('%Y-%m-%d %H:%M:%S')) + "\n")
        file.close()

def loop_through_stations(text):
    """
    loops through each of the stations in the dublin bikes static data
    stores relevant values in response to variables
    requests data from Open Weather Map API and import this data to the database
    """
    stations = json.loads(text)
    
    # loop through each dublin bikes station
    for station in stations:
        #print out name for debugging
        print(station.get("name"))

        # get dublin bikes variables
        station_number = station.get("number")
        station_lng = station.get("position").get("lng")
        station_lat = station.get("position").get("lat")
        
        # run OWM request and import to db
        r = requests.get(REQUEST, params={"lon": station_lng, "lat": station_lat, "appid": WAPIKEY})
        weather_to_db(r.text, station_number)


def weather_to_db(text, station_number):
    """
    Takes as input the response to OWM LAT LNG current status request
    Reads in all the values to a tuple and imports these to the database
    """
    weather = json.loads(text)
    # print(type(weather), len(weather))
    #print(weather)

    # added try except here as rain data is not always present
    try:
        rain_oneHour = weather.get("rain").get("1h")
    except:
        rain_oneHour = None
        print("no rain")

    vals = (weather.get("coord").get("lon"),
            weather.get("coord").get("lat"),
            weather.get("weather")[0].get("id"),
            weather.get("weather")[0].get("main"),
            weather.get("weather")[0].get("description"),
            weather.get("weather")[0].get("icon"),
            weather.get("base"),
            weather.get("main").get("temp"),
            weather.get("main").get("feels_like"),
            weather.get("main").get("temp_min"),
            weather.get("main").get("temp_max"),
            weather.get("main").get("pressure"),
            weather.get("main").get("humidity"),
            weather.get("visibility"),
            weather.get("wind").get("speed"),
            weather.get("wind").get("deg"),
            rain_oneHour,
            # rain_threeHour,
            weather.get("clouds").get("all"),
            weather.get("dt"),
            weather.get("sys").get("type"),
            weather.get("sys").get("id"),
            weather.get("sys").get("country"),
            weather.get("sys").get("sunrise"),
            weather.get("sys").get("sunset"),
            weather.get("timezone"),
            weather.get("id"),
            weather.get("name"),
            weather.get("cod"))
    # engine.execute("INSERT INTO `dublin_bikes`.`open_weather_south` values (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)", vals)

def main():
    # Database details
    DB_USER = os.environ.get("DB_USER")
    DB_PASS = os.environ.get("DB_PASS")
    DB_URL = os.environ.get("DB_URL")
    DB_PORT = os.environ.get("DB_PORT")

    # API Key for Dublin Bikes JCDecaux
    DubBike_API = os.environ.get("DUBBIKE_API")
    DubBike_NAME = "Dublin"
    DubBike_STATIONS = "https://api.jcdecaux.com/vls/v1/stations/"

    DubBike_API = os.environ.get("DUBBIKE_API")
    REQUEST = 'https://api.openweathermap.org/data/2.5/weather/'
    # Connect to database
    engine = create_engine("mysql+pymysql://{0}:{1}@{2}".format(DB_USER, DB_PASS, DB_URL), echo=True)

    failures = 0
    while True:
        try:
            now = datetime.datetime.now()
            print(WAPIKEY)
            DubBikes_r = requests.get(DubBike_STATIONS, params={"apiKey": DubBike_API, "contract": DubBike_NAME})
            loop_through_stations(DubBikes_r.text)
            time.sleep(5 * 60)
        except AttributeError as e:
            print(traceback.format_exc() + "\n ERROR: please stop the script and check for errors, request not generated correctly")
            if failures < 5: failures += 1
            error_log(e)
            time.sleep(failures * 30)
            print("got to here")
        except Exception as e:
            print(traceback.format_exc() + "\n ERROR: please stop the script and check for errors unknown error occured")
            if failures < 5: failures += 1
            error_log(e)
            time.sleep(failures * 30)


if __name__ == "__main__":
    main()


74571a8a54ecc764ebed7d0dc725ebe2
SMITHFIELD NORTH
Traceback (most recent call last):
  File "<ipython-input-11-9d1001fe3a72>", line 107, in main
    loop_through_stations(DubBikes_r.text)
  File "<ipython-input-11-9d1001fe3a72>", line 33, in loop_through_stations
    r = requests.get(REQUEST, params={"lon": station_lng, "lat": station_lat, "appid": WAPIKEY})
NameError: name 'REQUEST' is not defined

 ERROR: please stop the script and check for errors unknown error occured


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=71d679fb-aa0e-4eb1-84cc-5c32f888c0ea' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>